In [3]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "./code/files/enel1.pdf"
loader = PyPDFLoader(file_path)
pages = []
for page in loader.load():
    pages.append(page)

In [4]:
from pydantic import BaseModel,Field
from typing import  Literal

class EnergyCompanyResponse(BaseModel):
    periodo_referencia: str = Field(description="Periodo de referência da fatura no formato MM/YYYY")
    proxima_leitura: str = Field(description="Data da próxima leitura no formato DD/MM/YYYY")
    vencimento: str = Field(description="Data de vencimento da fatura no formato")
    bandeira_tarifaria: Literal['Vermelha','Amarela','Verde'] = Field(description="Bandeira tarifária da fatura")
    nome_distribuidora: str = Field(description="Nome da companhia distribuidora de energia")
    codigo_cliente: str = Field(description="Código do cliente na distribuidora")
    codigo_instalacao: str = Field(description="Código da instalação na distribuidora")
    tipo_fornecimento: Literal['Monofásico','Bifásico','Trifásico'] = Field(description="Tipo de fornecimento de energia")
    valor_total: float = Field(description="Valor total da fatura")
    consumo_kwh: float = Field(description="Consumo em kWh")
    client_nome: str = Field(description="Nome do cliente")
    
from langchain_core.output_parsers import JsonOutputParser
output_parser = JsonOutputParser(pydantic_object=EnergyCompanyResponse)

In [6]:
from langchain_openai import ChatOpenAI
#llm = ChatOllama(model="phi3.5",temperature=0)
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI()

In [7]:
from langchain_core.prompts import  PromptTemplate
prompt = PromptTemplate(
    template="""
                A resposta deve conter exclusivamente a informação em JSON, sem nenhum texto adicional.
                A resposta deve ser no formato e somente com os campos abaixo: \n
                {format_instructions}
                O texto da fatura é o seguinte: \n
                {context}
    """,
    input_variables=["context"],
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

In [8]:
from langchain.chains.question_answering import  load_qa_chain
qa_chain = load_qa_chain(llm=llm,chain_type="stuff",prompt=prompt,verbose=False)
response = qa_chain.run(input_documents=pages,question="Extraia as informações requisitadas da fatura de energia",output_parser=output_parser)
print(response)

/var/folders/mv/t88fcmds6cq7j7_hsq_34jnh0000gn/T/ipykernel_62689/4006064218.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm=llm,chain_type="stuff",prompt=prompt,verbose=False)
/var/folders/mv/t88fcmds6cq7j7_hsq_34jnh0000gn/T/ipykernel_62689/4006064218.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_cha

{
    "periodo_referencia": "05/2023",
    "proxima_leitura": "15/06/2023",
    "vencimento": "15/06/2023",
    "bandeira_tarifaria": "Verde",
    "nome_distribuidora": "Ampla Energia e Serviços S. A.",
    "codigo_cliente": "4680",
    "codigo_instalacao": "1847656",
    "tipo_fornecimento": "Trifásico",
    "valor_total": 338.45,
    "consumo_kwh": 320.91,
    "client_nome": "VICTOR DE AGUIAR NASCIMENTO"
}


In [9]:
output_parser.parse(response)

{'periodo_referencia': '05/2023',
 'proxima_leitura': '15/06/2023',
 'vencimento': '15/06/2023',
 'bandeira_tarifaria': 'Verde',
 'nome_distribuidora': 'Ampla Energia e Serviços S. A.',
 'codigo_cliente': '4680',
 'codigo_instalacao': '1847656',
 'tipo_fornecimento': 'Trifásico',
 'valor_total': 338.45,
 'consumo_kwh': 320.91,
 'client_nome': 'VICTOR DE AGUIAR NASCIMENTO'}